In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
# from yahist import Hist1D, Hist2D

In [2]:
# Utility function for FONLL
def getFONLLpt(filename):
    """
    Read a FONLL txt file and return arrays of 
    pt ds/dpt(central) ds/dpt(low)  ds/dpt(hi)
    in pb/GeV
    Also, integrate the cross-section
    (Integral is done in the stupidest way possible)
    Note: also works if FONLL file is ds/deta.           
    """
    
    data = np.loadtxt(filename)
    pt        = data[:,0]
    central   = data[:,1]
    down      = data[:,2]
    up        = data[:,3]

    sigma_central = central.sum()  *  (pt[1]-pt[0]) 
    sigma_down    = down.sum()     *  (pt[1]-pt[0]) 
    sigma_up      = up.sum()       *  (pt[1]-pt[0])
    return pt, [central, down, up], [sigma_central, sigma_down, sigma_up]

In [3]:
# read the FONLL files, store results, get x-sections
pt,   ds,    s    = getFONLLpt("FONLL/dsigma-dpt.txt")
eta,  dseta, seta = getFONLLpt("FONLL/dsigma-deta.txt")
pt28, ds28,  s28  = getFONLLpt("FONLL/dsigma-dpt-eta2.8.txt")
print("Inclusive FONLL cross section")
print('From ds/dpt:  sigma = {:.2e}'.format(s[0]), '+ {:.2e}'.format(s[2]-s[0]), '- {:.2e}'.format(s[0]-s[1]), 'pb')
print('From ds/deta: sigma = {:.2e}'.format(seta[0]), '+ {:.2e}'.format(seta[2]-seta[0]), '- {:.2e}'.format(seta[0]-seta[1]), 'pb')
print("-----------")
print("FONLL cross section abs(eta)<2.8")
print('From ds/dpt: sigma = {:.2e}'.format(s28[0]), '+ {:.2e}'.format(s28[2]-s28[0]), '- {:.2e}'.format(s28[0]-s28[1]), 'pb')
print("-----------")

# Now with a cut at pt > 5 GeV
ds_5    = ds   * (pt>5)    * (pt[1]-pt[0])
ds28_5  = ds28 * (pt28>5)  * (pt28[1]-pt28[0])
s_5     = [ds_5[0].sum(),   ds_5[1].sum(),   ds_5[2].sum()]  
s28_5   = [ds28_5[0].sum(), ds28_5[1].sum(), ds28_5[2].sum() ]
print("FONLL cross section pt>5")
print('From ds/dpt: sigma = {:.2e}'.format(s_5[0]), '+ {:.2e}'.format(s_5[2]-s_5[0]), '- {:.2e}'.format(s_5[0]-s_5[1]), 'pb')
print("-----------")
print("FONLL cross section pt> 5 and abs(eta)<2.8")
print('From ds/dpt: sigma = {:.2e}'.format(s28_5[0]), '+ {:.2e}'.format(s28_5[2]-s28_5[0]), '- {:.2e}'.format(s28_5[0]-s28_5[1]), 'pb')
print("-----------")

Inclusive FONLL cross section
From ds/dpt:  sigma = 4.74e+08 + 2.15e+08 - 1.86e+08 pb
From ds/deta: sigma = 4.66e+08 + 2.10e+08 - 1.80e+08 pb
-----------
FONLL cross section abs(eta)<2.8
From ds/dpt: sigma = 2.72e+08 + 1.20e+08 - 1.06e+08 pb
-----------
FONLL cross section pt>5
From ds/dpt: sigma = 1.89e+08 + 7.78e+07 - 5.61e+07 pb
-----------
FONLL cross section pt> 5 and abs(eta)<2.8
From ds/dpt: sigma = 1.33e+08 + 5.42e+07 - 4.05e+07 pb
-----------
